In [1]:
import os
import dill
import jsonlines
import pandas as pd
from utils import *

## 读取主诉尝试

In [ ]:
#读取主诉
import pandas as pd

# 设置路径
data_path = '/mnt/data2/MIMIC3/mimiciii/1.4'

# 读取 admissions 表
admissions = pd.read_csv(f'{data_path}/ADMISSIONS.csv')

# 选择主诉相关字段并查看前几条记录
print("=== ADMISSIONS 表中与主诉相关的字段 ===")
print(admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DIAGNOSIS']].head())


=== ADMISSIONS 表中与主诉相关的字段 ===
   SUBJECT_ID  HADM_ID            ADMITTIME ADMISSION_TYPE  \
0          22   165315  2196-04-09 12:26:00      EMERGENCY   
1          23   152223  2153-09-03 07:15:00       ELECTIVE   
2          23   124321  2157-10-18 19:34:00      EMERGENCY   
3          24   161859  2139-06-06 16:14:00      EMERGENCY   
4          25   129635  2160-11-02 02:06:00      EMERGENCY   

          ADMISSION_LOCATION  \
0       EMERGENCY ROOM ADMIT   
1  PHYS REFERRAL/NORMAL DELI   
2  TRANSFER FROM HOSP/EXTRAM   
3  TRANSFER FROM HOSP/EXTRAM   
4       EMERGENCY ROOM ADMIT   

                                           DIAGNOSIS  
0                            BENZODIAZEPINE OVERDOSE  
1  CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...  
2                                         BRAIN MASS  
3                     INTERIOR MYOCARDIAL INFARCTION  
4                            ACUTE CORONARY SYNDROME  


In [5]:
# 读取 noteevents 表（只读部分行以避免内存过载）
noteevents = pd.read_csv(f'{data_path}/NOTEEVENTS.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'TEXT'], nrows=10000)

# 过滤出可能包含主诉的类别，比如 'Discharge summary' 或 'Physician'
relevant_notes = noteevents[noteevents['CATEGORY'].isin(['Discharge summary', 'Physician'])]

# 查看前几条记录
print("\n=== NOTEEVENTS 表中可能包含主诉的记录 ===")
print(relevant_notes[['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'TEXT']].head(3))


=== NOTEEVENTS 表中可能包含主诉的记录 ===
   SUBJECT_ID  HADM_ID           CATEGORY  \
0       22532   167853  Discharge summary   
1       13702   107527  Discharge summary   
2       13702   167118  Discharge summary   

                                                TEXT  
0  Admission Date:  [**2151-7-16**]       Dischar...  
1  Admission Date:  [**2118-6-2**]       Discharg...  
2  Admission Date:  [**2119-5-4**]              D...  


In [14]:
import pandas as pd

# 设置数据路径
data_path = '/mnt/data2/MIMIC3/mimiciii/1.4'

# 只读取需要的列（减少内存消耗）
noteevents = pd.read_csv(f'{data_path}/NOTEEVENTS.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'TEXT'], nrows=5)

# 选择一条具体的笔记，例如第一条 discharge summary（你也可以换成 'Physician'）
example_note = noteevents[noteevents['CATEGORY'] == 'Discharge summary'].iloc[4]

# 显示这条笔记的内容
print("=== 一条 Discharge Summary 的完整 TEXT 字段 ===")
print(f"SUBJECT_ID: {example_note['SUBJECT_ID']}")
print(f"HADM_ID: {example_note['HADM_ID']}")
print(f"CATEGORY: {example_note['CATEGORY']}\n")
print(example_note['TEXT'])


=== 一条 Discharge Summary 的完整 TEXT 字段 ===
SUBJECT_ID: 26880
HADM_ID: 135453
CATEGORY: Discharge summary

Admission Date:  [**2162-3-3**]              Discharge Date:   [**2162-3-25**]

Date of Birth:  [**2080-1-4**]             Sex:   M

Service: MEDICINE

Allergies:
Patient recorded as having No Known Allergies to Drugs

Attending:[**First Name3 (LF) 1828**]
Chief Complaint:
Mr. [**Known lastname 1829**] was seen at [**Hospital1 18**] after a mechanical fall from
a height of 10 feet.  CT scan noted unstable fracture of C6-7 &
posterior elements.

Major Surgical or Invasive Procedure:
1. Anterior cervical osteotomy, C6-C7, with decompression and
excision of ossification of the posterior longitudinal ligament.
2. Anterior cervical deformity correction.
3. Interbody reconstruction.
4. Anterior cervical fusion, C5-C6-C7.
5. Plate instrumentation, C5-C6-C7.
6. Cervical laminectomy C6-C7, T1.
7. Posterior cervical arthrodesis C4-T1.
8. Cervical instrumentation C4-T1.
9. Arthrodesis augmentat

In [15]:
#随机提取20条
import pandas as pd

# 设置数据路径
data_path = '/mnt/data2/MIMIC3/mimiciii/1.4'

# 读取较大范围的数据以确保随机样本足够（建议 nrows ≥ 100000）
noteevents = pd.read_csv(
    f'{data_path}/NOTEEVENTS.csv',
    usecols=['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'TEXT'],
    nrows=10000  # 可调大以增加候选池
)

# 筛选出 'Discharge summary' 类型
discharge_notes = noteevents[noteevents['CATEGORY'] == 'Discharge summary']

# 随机抽样 20 条
sampled_notes = discharge_notes.sample(n=20, random_state=42)  # 设置 random_state 保证复现

# 保存为 Excel 文件
output_path = '/home/wuyicong/llm/Code-for-DDI/RAGents4DDI/experients/experient_rag/new_process_mimic_2506/discharge_summaries_sample_random20.xlsx'
sampled_notes.to_excel(output_path, index=False)

print(f"已随机选取 20 条 Discharge Summary 并导出到 Excel：{output_path}")


已随机选取 20 条 Discharge Summary 并导出到 Excel：/home/wuyicong/llm/Code-for-DDI/RAGents4DDI/experients/experient_rag/new_process_mimic_2506/discharge_summaries_sample_random20.xlsx


## 实验数据提取

In [2]:
class Voc(object):
    '''Define the vocabulary (token) dict'''

    def __init__(self):

        self.idx2word = {}
        self.word2idx = {}

    def add_sentence(self, sentence):
        '''add vocabulary to dict via a list of words'''
        for word in sentence:
            if word not in self.word2idx:
                self.idx2word[len(self.word2idx)] = word
                self.word2idx[word] = len(self.word2idx)

# create voc set
def create_str_token_mapping(df, vocabulary_file):
    diag_voc = Voc()
    med_voc = Voc()
    pro_voc = Voc()

    for index, row in df.iterrows():
        diag_voc.add_sentence(row["ICD9_CODE"])
        med_voc.add_sentence(row["ATC3"])
        pro_voc.add_sentence(row["PRO_CODE"])

    dill.dump(
        obj={"diag_voc": diag_voc, "med_voc": med_voc, "pro_voc": pro_voc},
        file=open(vocabulary_file, "wb"),
    )
    return diag_voc, med_voc, pro_voc

In [3]:
tokenizer = dill.load(open("./handled/voc_final.pkl", "rb"))

## Step 1:
Preprocess the raw MIMIC-III data as the original medication recommendation works

In [3]:
base_dir = "/home/wuyicong/llm/Code-for-DDI/auxiliary"   # base folder
output_base_dir = ""

## Some auxiliary info, such as DDI, ATC and ICD
RXCUI2atc4_file = os.path.join(base_dir, "RXCUI2atc4.csv")
cid2atc6_file = os.path.join(base_dir, "drug-atc.csv")
ndc2RXCUI_file = os.path.join(base_dir, "ndc2RXCUI.txt")
ddi_file = os.path.join(base_dir, "drug-DDI.csv")
drugbankinfo = os.path.join(base_dir, "drugbank_drugs_info.csv")

读取和处理 MIMIC-III 数据集中的 PRESCRIPTIONS.csv（药品处方）、DIAGNOSES_ICD.csv（诊断数据）、PROCEDURES_ICD.csv（医疗程序数据）等。

In [4]:
mimic3_dir = "/mnt/data2/MIMIC3/mimiciii/1.4"
med_file = os.path.join(mimic3_dir, "PRESCRIPTIONS.csv")
diag_file = os.path.join(mimic3_dir, "DIAGNOSES_ICD.csv")
procedure_file = (
    os.path.join(mimic3_dir, "PROCEDURES_ICD.csv")
)

# input auxiliary files
med_structure_file = os.path.join(output_base_dir, "./handled/atc32SMILES.pkl")

# output files
ddi_adjacency_file = os.path.join(output_base_dir, "./handled/full/ddi_A_final.pkl")
ehr_adjacency_file = os.path.join(output_base_dir, "./handled/full/ehr_adj_final.pkl")
ehr_sequence_file = os.path.join(output_base_dir, "./handled/full/records_final.pkl")
vocabulary_file = os.path.join(output_base_dir, "./handled/full/voc_final.pkl")
ddi_mask_H_file = os.path.join(output_base_dir, "./handled/full/ddi_mask_H.pkl")
atc3toSMILES_file = os.path.join(output_base_dir, "./handled/full/atc3toSMILES.pkl")

In [8]:
print(med_file)

/mnt/data2/MIMIC3/mimiciii/1.4/PRESCRIPTIONS.csv


读取处方数据，并通过 med_process 处理它。
根据药品编码（NDC）映射到 ATC 分类。
选择常见的药品（过滤药品数据）并处理它们的 SMILES 表示。

In [ ]:
# for med
med_pd = med_process(med_file)  # process the raw file
# med_pd_lg2 = process_visit_lg1(med_pd).reset_index(drop=True)   # remain the single-visit
med_pd_lg2 = process_visit_lg2(med_pd).reset_index(drop=True)   # filter out the patient has less 2 visits
med_pd = med_pd.merge(
    med_pd_lg2[["SUBJECT_ID"]], on="SUBJECT_ID", how="inner"
).reset_index(drop=True)

med_pd = codeMapping2atc4(med_pd, ndc2RXCUI_file, RXCUI2atc4_file)
med_pd = filter_300_most_med(med_pd)

# med to SMILES mapping 
# 药物信息从 drugbank_drugs_info.csv 读取，然后通过 ATC3toDrug 类进行药物分类。 将药物的 ATC3 代码映射到 SMILES 结构式表示，并将结果保存在文件中。
atc3toDrug = ATC3toDrug(med_pd)
druginfo = pd.read_csv(drugbankinfo)
atc3toSMILES = atc3toSMILES(atc3toDrug, druginfo)
dill.dump(atc3toSMILES, open(atc3toSMILES_file, "wb"))
med_pd = med_pd[med_pd.ATC3.isin(atc3toSMILES.keys())]
print("complete medication processing")

# for diagnosis 读取并处理诊断数据
diag_pd = diag_process(diag_file)

print("complete diagnosis processing")

# for procedure 读取并处理医疗程序数据
pro_pd = procedure_process(procedure_file)
# pro_pd = filter_1000_most_pro(pro_pd)

print("complete procedure processing")

# combine 将药品、诊断和程序数据合并成一个大表格 data
data = combine_process(med_pd, diag_pd, pro_pd)
print("complete combining")

/home/wuyicong/llm/Code-for-DDI/LEADER-pytorch-master/data/mimic3/utils.py:28: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  med_pd = pd.read_csv(med_file, dtype={"NDC": "category"})
/home/wuyicong/llm/Code-for-DDI/LEADER-pytorch-master/data/mimic3/utils.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  med_pd.fillna(method="pad", inplace=True)
/tmp/ipykernel_307307/1130413577.py:14: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  druginfo = pd.read_csv(drugbankinfo)


complete medication processing
complete diagnosis processing
complete procedure processing
complete combining


In [6]:
statistics(data)

#patients  (6350,)
#clinical events  15032
#diagnosis  1959
#med  112
#procedure 1430
#avg of diagnoses  10.508315593400745
#avg of medicines  11.647751463544438
#avg of procedures  3.8436668440659925
#avg of vists  2.367244094488189
#max of diagnoses  128
#max of medicines  64
#max of procedures  50
#max of visit  29


In [8]:
# # create vocab
# diag_voc, med_voc, pro_voc = create_str_token_mapping(data, vocabulary_file)
# print("obtain voc")

# # create ehr sequence data
# records = create_patient_record(data, diag_voc, med_voc, pro_voc, ehr_sequence_file)
# print("obtain ehr sequence data")

# # create ddi adj matrix
# ddi_adj = get_ddi_matrix(records, med_voc, ddi_file, cid2atc6_file, ehr_adjacency_file, ddi_adjacency_file)
# print("obtain ddi adj matrix")

# # get ddi_mask_H
# ddi_mask_H = get_ddi_mask(atc3toSMILES, med_voc)
# dill.dump(ddi_mask_H, open(ddi_mask_H_file, "wb"))

## Step 2: Get side info
Extract side information of patients from other csv ，通过患者的 HADM_ID 将入院信息（如保险、语言、宗教、婚姻状态等）添加到数据中。

In [7]:
def get_side(source_df, side_df, side_columns, aligh_column):

    side_df = side_df[side_columns]
    source_df = pd.merge(source_df, side_df, how="left", on=aligh_column)

    return source_df

In [8]:
admission = pd.read_csv("/mnt/data2/MIMIC3/mimiciii/1.4/ADMISSIONS.csv")
data = get_side(data, admission, 
                ["HADM_ID", "INSURANCE", "LANGUAGE", "RELIGION", "MARITAL_STATUS", "ETHNICITY", "DIAGNOSIS"],
                "HADM_ID"
                )

In [9]:
data.fillna(value="unknown", inplace=True)

## Step 3: Map ATC to drugname
Resolve the mapping. In the original preprocessed data, the drug is represented by ATC code, but we need the drugname for LLM.


In [10]:
RXCUI2atc4 = pd.read_csv(RXCUI2atc4_file) #读取 RXCUI2atc4.csv 文件，映射药品的 NDC（国家药品编码）与 ATC4（药物分类代码）
RXCUI2atc4["NDC"] = RXCUI2atc4["NDC"].map(lambda x: x.replace("-", ""))
with open(ndc2RXCUI_file, "r") as f:
    ndc2RXCUI = eval(f.read())

In [11]:
RXCUI2ndc = dict(zip(ndc2RXCUI.values(), ndc2RXCUI.keys()))
RXCUI2atc4["RXCUI"] = RXCUI2atc4["RXCUI"].astype("str")
RXCUI2atc4["NDC"] = RXCUI2atc4["RXCUI"].map(RXCUI2ndc)
RXCUI2atc4.dropna(axis=0, how="any", inplace=True)
RXCUI2atc4.drop_duplicates(inplace=True)

In [12]:
RXCUI2atc4.shape, RXCUI2atc4.nunique()

((32732, 5),
 YEAR       73
 MONTH      12
 NDC      2037
 RXCUI    2037
 ATC4      445
 dtype: int64)

In [13]:
RXCUI2atc4.drop_duplicates(inplace=True)
RXCUI2atc4.shape

(32732, 5)

In [14]:
med_pd.head(5)

,SUBJECT_ID,HADM_ID,STARTDATE,DRUG,ATC3
0,17,161087,2135-05-09,Acetaminophen,N02B
1,17,161087,2135-05-09,Aspirin EC,A01A
2,17,161087,2135-05-09,Aspirin,A01A
3,17,161087,2135-05-09,Ranitidine,A02B
4,17,161087,2135-05-09,Docusate Sodium,A06A


In [15]:
med_pd = pd.read_csv(med_file, dtype={"NDC": "category"})
med_pd.head(5)

/tmp/ipykernel_307307/1537339614.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  med_pd = pd.read_csv(med_file, dtype={"NDC": "category"})


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [16]:
med_pd.head(5)

,SUBJECT_ID,HADM_ID,STARTDATE,DRUG,ATC3
0,17,161087,2135-05-09,Acetaminophen,N02B
1,17,194023,2134-12-27,Acetaminophen,N02B
2,21,111970,2135-02-06,Acetaminophen,N02B
3,23,152223,2153-09-03,Acetaminophen,N02B
4,36,122659,2131-05-15,Acetaminophen,N02B


In [17]:
med_pd = pd.read_csv(med_file, dtype={"NDC": "category"})
med_pd.head(5)

<ipython-input-17-c97a9618a25a>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  med_pd = pd.read_csv(med_file, dtype={"NDC": "category"})


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [16]:
med_pd["NDC"].astype("str")
med_pd = pd.merge(med_pd, RXCUI2atc4, how="left", on="NDC")

In [18]:
atc2drug = pd.read_csv("/home/wuyicong/llm/Code-for-DDI/auxiliary/WHO ATC-DDD 2021-12-03.csv")
atc2drug["code_len"] = atc2drug["atc_code"].map(lambda x: len(x))
atc2drug = atc2drug[atc2drug["code_len"]==4]    # all levels are included. We only need the 4th level, i.e., ATC4
atc2drug.rename(columns={"atc_code": "ATC4"}, inplace=True)
atc2drug.drop(columns=["ddd", "uom", "adm_r", "note", "code_len"], axis=1, inplace=True)

In [19]:
atc2drug.head(10)

,ATC4,atc_name
2,A01A,STOMATOLOGICAL PREPARATIONS
46,A02A,ANTACIDS
79,A02B,DRUGS FOR PEPTIC ULCER AND GASTRO-OESOPHAGEAL ...
147,A02X,OTHER DRUGS FOR ACID RELATED DISORDERS
149,A03A,DRUGS FOR FUNCTIONAL GASTROINTESTINAL DISORDERS
219,A03B,"BELLADONNA AND DERIVATIVES, PLAIN"
235,A03C,ANTISPASMODICS IN COMBINATION WITH PSYCHOLEPTICS
255,A03D,ANTISPASMODICS IN COMBINATION WITH ANALGESICS
267,A03E,ANTISPASMODICS AND ANTICHOLINERGICS IN COMBINA...
270,A03F,PROPULSIVES


In [20]:
RXCUI2atc4["ATC4"] = RXCUI2atc4["ATC4"].map(lambda x: x[:4])

In [21]:
# all atc code in original data can be mapped to drugname by atc2drug.
# means that we use the same data as the traditional medication recommendation models.
pd.merge(RXCUI2atc4, atc2drug, on="ATC4", how="left")["atc_name"].isna().sum()

0

In [22]:
atc2drug["atc_name"] = atc2drug["atc_name"].map(lambda x: x.lower())

In [23]:
# get the atc2drug and drug2atc mapping dict
atc2drug_dict = dict(zip(atc2drug["ATC4"].values, atc2drug["atc_name"].values))
drug2atc_dict = dict(zip(atc2drug["atc_name"].values, atc2drug["ATC4"].values))

In [25]:
# import json
# json.dump({"atc2drug": atc2drug_dict, "drug2atc": drug2atc_dict}, open("./handled/full_atc2drug.json", "w"))

In [25]:
# get the diagnosis and procedure mapping dict, which both use the ICD. these mappings are in raw MIMIC dataset
icd2diag = pd.read_csv("/mnt/data2/MIMIC3/mimiciii/1.4/D_ICD_DIAGNOSES.csv")
icd2diag_dict = dict(zip(icd2diag["ICD9_CODE"].astype(str).values, icd2diag["SHORT_TITLE"].values))

In [26]:
icd2proc = pd.read_csv("/mnt/data2/MIMIC3/mimiciii/1.4/D_ICD_PROCEDURES.csv")
icd2proc_dict = dict(zip(icd2proc["ICD9_CODE"].astype(str).values, icd2proc["SHORT_TITLE"].values))

In [27]:
def decode(code_list, decoder):
    # decode a list of code into corresponding names
    miss_match = 0
    target_list = []
    for code in code_list:
        try:
            target_list.append(decoder[code])
        except:
            miss_match += 1
    
    #print(miss_match)

    return target_list

In [28]:
data["drug"] = data["ATC3"].map(lambda x: decode(x, atc2drug_dict))
data["diagnosis"] = data["ICD9_CODE"].map(lambda x: decode(x, icd2diag_dict))
data["procedure"] = data["PRO_CODE"].map(lambda x: decode(x, icd2proc_dict))

some miss matches occurs in diagnosis and procedures, but no for drug

In [29]:
data.iloc[1]["PRO_CODE"]

['3571', '3961', '8872']

In [30]:
def profile_tokenization(df, profile_columns):
    prof_dict = {"word2idx":{}, "idx2word": {}}
    for prof in profile_columns:
        prof_dict["idx2word"][prof] = dict(zip(range(df[prof].nunique()), df[prof].unique()))
        prof_dict["word2idx"][prof] = dict(zip(df[prof].unique(), range(df[prof].nunique())))
    return prof_dict

In [32]:
# profile_dict = profile_tokenization(data, ["INSURANCE", "LANGUAGE", "RELIGION", "MARITAL_STATUS", "ETHNICITY"])
# json.dump(profile_dict, open("./handled/full_profile_dict.json", "w"))

## Step 4: Construct Prompt
Design the prompt templates and construct the prompt

In [31]:
# prompt templates
main_template = "The patient has <VISIT_NUM> times ICU visits. \n <HISTORY> In this visit, he has diagnosis: <DIAGNOSIS>; procedures: <PROCEDURE>. Then, the patient should be prescribed: "
hist_template = "In <VISIT_NO> visit, the patient had diagnosis: <DIAGNOSIS>; procedures: <PROCEDURE>. The patient was prescribed drugs: <MEDICATION>. \n"

In [34]:
# add some patient's profiles
# main_template = "The patient's insurance type is <INSU>, language is <LANG>, religion is <RELIGION>, marital status is <MARITAL>, ethnicity is <ETHN>. The patient has <VISIT_NUM> times ICU visits. \n <HISTORY> In this visit, he has diagnosis: <DIAGNOSIS>; procedures: <PROCEDURE>. Then, the patient should be prescribed: "

In [32]:
def concat_str(str_list):
    # concat a list of drug / diagnosis / procedures
    target_str = ""
    for meta_str in str_list:
        target_str = target_str + meta_str + ", "
    target_str = target_str[:-2]    # remove the last comma

    return target_str

In [33]:
llm_data = []

for subject_id in data["SUBJECT_ID"].unique():
    item_df = data[data["SUBJECT_ID"] == subject_id]
    visit_num = item_df.shape[0] - 1
    patient = []

    profile = item_df.iloc[0]
    patient_str = main_template.replace("<INSU>", profile["INSURANCE"].lower())\
                               .replace("<LANG>", profile["LANGUAGE"].lower())\
                               .replace("<RELIGION>", profile["RELIGION"].lower())\
                               .replace("<MARITAL>", profile["MARITAL_STATUS"].lower())\
                               .replace("<ETHN>", profile["ETHNICITY"].lower())

    patient_profile = {"INSURANCE": profile["INSURANCE"], "LANGUAGE": profile["LANGUAGE"],
                       "RELIGION": profile["RELIGION"], "MARITAL_STATUS": profile["MARITAL_STATUS"],
                       "ETHNICITY": profile["ETHNICITY"]}

    # get each historical visit string
    for visit_no, (_, row) in enumerate(item_df.iterrows()):
        drug, diag, proc = concat_str(row["drug"]), concat_str(row["diagnosis"]), concat_str(row["procedure"])
        patient.append(hist_template.replace("<VISIT_NO>", str(visit_no+1))\
                                    .replace("<DIGNOSIS>", diag)\
                                    .replace("<PROCEDURE>", proc)\
                                    .replace("<MEDICATION>", drug))
    patient.pop()   # remove the ground truth record

    # filter out the patients with more than N times visits
    if len(patient) > 3:
        patient = patient[-3:]

    # concat all historical visit strings and get hist strings
    hist_str = ""
    for meta_hist in patient:
        hist_str += meta_hist
    
    patient_str = patient_str.replace("<VISIT_NUM>", str(visit_num))\
                             .replace("<HISTORY>", hist_str)\
                             .replace("<DIAGNOSIS>", diag)\
                             .replace("<PROCEDURE>", proc)
    
    drug_code = [str(x) for x in row["ATC3"]]

    hist = {"diagnosis": [], "procedure": [], "medication": []}
    for _, row in item_df.iterrows():
        hist["diagnosis"].append([str(x) for x in row["ICD9_CODE"]])
        hist["procedure"].append([str(x) for x in row["PRO_CODE"]])
        hist["medication"].append([str(x) for x in row["ATC3"]])
        
    llm_data.append({"input": patient_str, "target": drug, 
                     "subject_id": int(subject_id), "drug_code": drug_code,
                     "records": hist, "profile": patient_profile})
        

In [34]:
file_path = "./handled/"

def read_data(data_path):
    '''read data from jsonlines file'''
    data = []

    with jsonlines.open(file_path + data_path, "r") as f:
        for meta_data in f:
            data.append(meta_data)

    return data


def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open(file_path + data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

In [35]:
# split the dataset: 8:1:1
train_split = int(len(llm_data) * 0.8)
val_split = int(len(llm_data) * 0.1)
train = llm_data[:train_split]
val = llm_data[train_split:train_split+val_split]
test = llm_data[train_split+val_split:]

In [36]:
# cut too long sequences
save_data("train_0105.json", train)
save_data("val_0105.json", val)
save_data("test_0105.json", test)